# Da TimeSeries a Eventi

In questo file sono state implementate delle funzioni che permettono di passare da un dataframe basato su **timeseries** ad un dataframe contenente degli **eventi**.<br>
Ho deciso di concentrare la mia analisi su tre file:
- **calories.json**
- **sleep.json**
- **exercise.json.**<br>

Ho preso questa decisione per vedere se ci sono correlazioni tra i dati di questi dataset.

In [ ]:
import nbimporter
from t1_ETLBasics import calories_to_df, sleep_to_df
from t1_ETLBasics import exercise_to_df
import datetime
import dateutil as du
PATH = '../../pmdata/'

## Calories
Da timeseries passiamo ad eventi. 
- prima raggruppo per partecipante e ora e calcolo media, deviazione,standard e somma.


In [ ]:
cal = calories_to_df(PATH,[1,5])

In [ ]:
cal

La seguente funzione prende in input il dataframe delle calorie e dopo aver raggruppato per **partecipante** e **ora** crea degli eventi.

In [ ]:
def calories_to_events(df):
    """funzione che trasforma il df delle calorie da 
        timeseries a eventi.Raggruppa per partecipante e ora e
        calcola media, deviazione standard e somma delle calorie.
        Crea degli eventi.

        - parametro df: dataframe da mpdificare
        - ritorna cal_agg: dataframe aggregato in cui """
    df['pid'] = [x[0] for x in list(df.index)]
    df['ts'] = [x[1] for x in list(df.index)]
    df['hour'] = df['ts'].apply(lambda x: x.strftime('%Y-%m-%d %H'))
    cal_agg = df.groupby(['pid','hour']).agg({'calories': ['mean','std','sum']})
    cal_agg['c_mean'] = cal_agg['calories']['mean']
    cal_agg['c_std'] = cal_agg['calories']['std']
    cal_agg['c_sum'] = cal_agg['calories']['sum']
    cal_agg['ts'] = [x[1] for x in list(cal_agg.index)]
    cal_agg = cal_agg.drop(['calories'] , axis=1)
    cal_agg['date'] = cal_agg['ts'].apply(lambda x: du.parser.parse(x[0:-3]))
    cal_agg['hour'] = cal_agg['ts'].apply(lambda x: int(x[-2:]))
    cal_agg['eight_part_day'] = cal_agg['hour'].apply(lambda x: int(x/3))
    cal_agg['quarter_day'] = cal_agg['hour'].apply(lambda x: int(x/6))
    cal_agg['half_day'] = cal_agg['hour'].apply(lambda x: int(x/12))
    cal_agg['week_day'] = cal_agg['date'].apply(lambda x: x.strftime('%A'))
    cal_agg['weekend'] = cal_agg['week_day'].apply(lambda x: True if x in ['Saturday','Sunday'] else False)
    cal_agg['id'] = [x[0] for x in list(cal_agg.index)]
    cal_agg = cal_agg.drop(['date'], axis=1)
    return cal_agg

In [ ]:
cal_events = calories_to_events(cal)

In [ ]:
#cal_events.to_csv('./dataframe/cal_events.csv')
cal_events

## Sleep

In [ ]:
sl = sleep_to_df(PATH,[1])

In [ ]:
sl

La seguente funzione prende in input il dataframe del sleep.json e crea degli eventi.

In [ ]:
def sleep_to_events(df):
    """ Funzione che crea delle etichette dal dataframe sleep.
    
    -parametro df: dataframe sleep
    -ritorna df: dataframe aggiornato con gli eventi in aggiunta."""
    df['pid'] = [x[0] for x in list(df.index)]
    df['startT'] = [str(x)[0:13] for x in df['startT']]
    df['startT'] = [datetime.datetime.strptime(x, '%Y-%m-%d %H') for x in df['startT']]
    #df['ts'] = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H') for x in df['startT']]
    df['ts'] = df['startT'].apply(lambda x: x.strftime('%Y-%m-%d %H'))
    df['day'] = [datetime.datetime.strptime(str(x[1]), '%Y-%m-%d') for x in list(df.index)]
    df['startSleepH'] = [str(x)[11:13] for x in df['startT']]
    df['endSleepH'] = [str(x)[11:13] for x in df['endT']]
    df['durationH'] = [round((float(x)/(1000*60*60))%24) for x in df['duration']]
    df['week_day'] = df['day'].apply(lambda x: x.strftime('%A'))
    df['weekend']= df['week_day'].apply(lambda x: True if x in ['Saturday', 'Sunday'] else False)
    df = df.drop(['startT','endT','type','timeInBed','shortData','data','levelDeep','levelWake','levelLight','levelRem'],axis = 1)
    return df
    

In [ ]:
sleep = sleep_to_events(sl)

In [ ]:
#sleep.to_csv('./dataframe/sleep.csv')
sleep

## Exercise

In [ ]:
ex = exercise_to_df(PATH,[1])

In [ ]:
ex

Funzione che crea etichette per evidenziare degli eventi per il dataframe exercise.

In [ ]:
def exercise_to_events(df):
    """Funzione che prende un dataframe e lo trasforma andando 
    a creare delle etichette per creare degli eventi.
    
    parametro df: dataframe di exercise.json
    ritorna: dataframe in cui ho gli eventi e i valori"""
    df['pid'] = [x[0] for x in list(df.index)]
    df['date'] = [x[1].strftime("%Y-%m-%d") for x in list(df.index)]
    df['startHour'] = [x[1].strftime("%H") for x in list(df.index)]
    df['ts'] = [x[1] for x in list(df.index)]
    df['extendHour'] = df['ts'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
    df['ts'] = df['ts'].apply(lambda x: x.strftime('%Y-%m-%d %H'))
    df['durationMin'] = df['duration'].apply(lambda x: round(float(x)/(1000*60))%60)
    arr = []
    for e in df['startHour']:
        if(int(e) >= 5 and int(e)<= 12):
            arr.append('Mattina')
        elif(int(e) >12 and int(e)<= 17):
            arr.append('Pomeriggio')
        elif(int(e) >17 and int(e)<= 22):
            arr.append('Sera')
        else:
            arr.append('Notte')
    df['part_day'] = arr
    #df = df.drop(['ts'],axis=1)
    return df[['pid','date','startHour','extendHour','steps','durationMin','activityName','calories','ts','part_day']]

In [ ]:
exercise = exercise_to_events(ex)

In [ ]:
#exercise.to_csv('./dataframe/exercise_events.csv')
exercise